In [1]:
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  

import tools.Sample_Tools as smpl
import tools.Pretreat_Tools as pretreat
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)

from base.JuUnits import parallal_task,task_chunk_split
from base.JuUnits import excute_for_multidates

import ind.Ind_MOM_Effect_Coin as mom


%load_ext autoreload
%autoreload 2
%aimport ind.Ind_MOM_Effect_Coin



In [ ]:
# #读取已经缓存的复权数据，并预处理
tail = True
origin_start = None
if tail:
    df_tail = load_cache('all_tail_qfq',cache_type=CACHE_TYPE.STOCK).sort_index()
    df_train = load_cache('all_train_qfq',cache_type=CACHE_TYPE.STOCK).sort_index()
    origin_start = df_tail.index.get_level_values(0).min()
    year_data_reserve = origin_start.year - 2

    df_all = pd.concat([df_train.loc[df_train.index.get_level_values(0) >= str(year_data_reserve)],df_tail],axis=0)
else:
    df_all = load_cache('all_train_qfq',cache_type=CACHE_TYPE.STOCK).sort_index()
    
smpl.optimize_data_type(df_all)


df_tail = None
df_train = None

#############测试#######################
# ['000001','000008','600600','000729']
# stock_df = stock_df.loc[(slice(None),stock_df.index.get_level_values(1).unique().to_list()[0:11]),:]
# ret_t, ret_t_excess, market_value_t, ret_excess_market_t = base10.prepare_data(stock_df,ret_fs_daily)

stock_df=df_all
stock_codes = stock_df.index.get_level_values(1).unique().to_list()
np.random.shuffle(stock_codes)

curret = smpl.get_current_return(stock_df,'close')
turnover = stock_df['volume'] / (stock_df['lshares'] *100) # 手/万股


def generate_factors(codes, data_df=None, cur_ret=None, turn_over=None):
    import ind.Ind_MOM_Effect_Coin as mom
    from base.JuUnits import excute_for_multidates
    import numpy as np
    import pandas as pd

    df = data_df.loc[(slice(None),codes),:]
    cur_ret_ = cur_ret.loc[(slice(None),codes)]
    turn_over_ = turn_over.loc[(slice(None),codes)]
        
    
    interday_fluctuation_reverse = mom.get_interday_fluctuation_reverse(df, cur_ret_, turn_over_)
    interday_turnover_reverse = mom.get_interday_turnover_reverse(df, cur_ret_, turn_over_)


    intraday_fluctuation_reverse = mom.get_intraday_fluctuation_reverse(df, turn_over_)
    intraday_turnover_reverse = mom.get_intraday_turnover_reverse(df, turn_over_)


    overnight_fluctuation_reverse = mom.get_overnight_fluctuation_reverse(df)
    overnight_turnover_reverse = mom.get_overnight_turnover_reverse(df, turn_over_)

    
    
    return pd.concat([interday_fluctuation_reverse, interday_turnover_reverse, 
                      intraday_fluctuation_reverse, intraday_turnover_reverse,
                      overnight_fluctuation_reverse,overnight_turnover_reverse
                     ], axis=1).loc[(slice(None),codes),:]

worker=6
task = task_chunk_split(stock_codes, worker)
results = parallal_task(worker, generate_factors, task, 
                        data_df=stock_df, cur_ret=curret, turn_over=turnover)

results = pd.concat(results).sort_index()
results.index.names=['date','code']


if tail:
    save_cache('mom_effect_coin_tail', results.loc[origin_start:], cache_type=CACHE_TYPE.FACTOR)
else:
    save_cache('mom_effect_coin_train', results, cache_type=CACHE_TYPE.FACTOR)

Now in the main code. Process name is: base.JuUnits
base.JuUnits, subpid:10548  pid:9696


  0%|          | 0/6 [00:00<?, ?it/s]

In [17]:
results

interday_fluctuation_reverse  interday_turnover_reverse  \
date       code                                                              
2019-01-02 000001                           NaN                        NaN   
           000002                           NaN                        NaN   
           000004                           NaN                        NaN   
           000006                           NaN                        NaN   
           000007                           NaN                        NaN   
...                                         ...                        ...   
2022-12-30 605580                      0.001750                   0.000571   
           605588                      0.005206                   0.007065   
           605589                     -0.005678                  -0.001319   
           605598                     -0.008837                   0.001409   
           605599                     -0.002318                   0.004401   

                   intraday_fluctuation_reverse  intraday_turnover_reverse  \
date       code                                                              
2019-01-02 000001                           NaN                        NaN   
           000002                           NaN                        NaN   
           000004                           NaN                        NaN   
           000006                           NaN                        NaN   
           000007                           NaN                        NaN   
...                                         ...                        ...   
2022-12-30 605580                      0.000995                   0.000454   
           605588                      0.005670                   0.007289   
           605589                     -0.005878                  -0.003740   
           605598                     -0.013201                   0.000683   
           605599                     -0.001854                   0.002683   

                   overnight_fluctuation_reverse  overnight_turnover_reverse  
date       code                                                               
2019-01-02 000001                            NaN                         NaN  
           000002                            NaN                         NaN  
           000004                            NaN                         NaN  
           000006                            NaN                         NaN  
           000007                            NaN                         NaN  
...                                          ...                         ...  
2022-12-30 605580                      -0.004097                    0.000143  
           605588                      -0.005949                   -0.000208  
           605589                       0.003694                    0.002392  
           605598                       0.016165                    0.000683  
           605599                      -0.006002                    0.001722  

[2797124 rows x 6 columns]

In [18]:
results.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2797124 entries, (Timestamp('2019-01-02 00:00:00'), '000001') to (Timestamp('2022-12-30 00:00:00'), '605599')
Data columns (total 6 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   interday_fluctuation_reverse   float64
 1   interday_turnover_reverse      float64
 2   intraday_fluctuation_reverse   float64
 3   intraday_turnover_reverse      float64
 4   overnight_fluctuation_reverse  float64
 5   overnight_turnover_reverse     float64
dtypes: float64(6)
memory usage: 138.8+ MB


In [19]:
x = load_cache('mom_effect_coin_tail', cache_type=CACHE_TYPE.FACTOR)
x

interday_fluctuation_reverse  interday_turnover_reverse  \
date       code                                                              
2021-01-04 000001                      0.000618                   0.000441   
           000002                      0.003768                   0.001509   
           000004                     -0.010568                  -0.001874   
           000006                      0.004119                   0.011896   
           000007                     -0.012189                   0.003488   
...                                         ...                        ...   
2022-12-30 605580                      0.001750                   0.000571   
           605588                      0.005206                   0.007065   
           605589                     -0.005678                  -0.001319   
           605598                     -0.008837                   0.001409   
           605599                     -0.002318                   0.004401   

                   intraday_fluctuation_reverse  intraday_turnover_reverse  \
date       code                                                              
2021-01-04 000001                     -0.002241                   0.001819   
           000002                      0.004021                   0.000951   
           000004                     -0.007800                  -0.003710   
           000006                      0.005945                   0.010515   
           000007                     -0.003900                  -0.005649   
...                                         ...                        ...   
2022-12-30 605580                      0.000995                   0.000454   
           605588                      0.005670                   0.007289   
           605589                     -0.005878                  -0.003740   
           605598                     -0.013201                   0.000683   
           605599                     -0.001854                   0.002683   

                   overnight_fluctuation_reverse  overnight_turnover_reverse  
date       code                                                               
2021-01-04 000001                      -0.003189                   -0.001390  
           000002                      -0.004212                    0.000532  
           000004                       0.011763                    0.001691  
           000006                      -0.004182                    0.001421  
           000007                      -0.002965                    0.009203  
...                                          ...                         ...  
2022-12-30 605580                      -0.004097                    0.000143  
           605588                      -0.005949                   -0.000208  
           605589                       0.003694                    0.002392  
           605598                       0.016165                    0.000683  
           605599                      -0.006002                    0.001722  

[1456721 rows x 6 columns]